# Vectorizacion de texto

# Funciones de limpieza

In [1]:
# Importo las librerias necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
import json
import glob
import re
from pprint import pprint

In [2]:
# Funciones de limpieza de títulos

def remove_stops(text, stops):
    text = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.) ]+','',text)
    text = re.sub(r'[^\w\s -]','',text)
    text = re.sub(r'-',' ',text)
    text = text.lower()
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

def clean_docs(docs):
    stops = stopwords.words("spanish")
    final = []
    for doc in docs:
        #clean_doc = remove_stops(doc, stops)
        final.append(doc)
    return (final)

# Creo las variables con titulos candidatos y encontrados.

In [3]:
# Creo las variables con titulos candidatos y encontrados.

encontrado = pd.read_csv("titulosEncontrados.csv", sep=',',  encoding='utf-8')
candidato = pd.read_csv("titulosCandidatos.csv", sep=',',  encoding='utf-8')
candidato = candidato.dropna()
encontrado = encontrado.dropna()
titEncontrado = clean_docs(encontrado['titulo'])
titCandidato = clean_docs(candidato['titulo'])

In [4]:
# Creo las variables que contienen los dataframes de titulos candidatos y encontrados.
dfTitCandidato = pd.read_csv('df_titulosCandidatos.csv', sep=',',  encoding='utf-8')
dfTitEncontrado = pd.read_csv('df_titulosEncontrados.csv', sep=',',  encoding='utf-8')
dfExpTitulosCandidatos = pd.read_csv('df_ExpTitulosCandidatos.csv', sep=',',  encoding='utf-8')

# TF_IDF

In [5]:
stops = stopwords.words("spanish")
# Creo el vectorizador tf-idf de titulos encontrados y transformo titulos candidatos y encontrados
vectorizer = TfidfVectorizer(
                                lowercase=False,
                                use_idf=True,
                                max_features=20000,
                                max_df=0.99,
                                min_df=50,
                                ngram_range = (1,3),
                                stop_words = stops
                            )

vectorizer.fit(titEncontrado)
vectorsCandidatos = vectorizer.transform(titCandidato)
vectorsEncontrado = vectorizer.transform(titEncontrado)
feature_names = vectorizer.get_feature_names_out()

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(vectorsCandidatos, vectorsEncontrado)

In [7]:
# TOPIC MODELING - K-MEANS
true_k = 20

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=100, n_init=1)

model.fit(vectorsEncontrado)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

with open ("clusters_results.txt", "w", encoding="utf-8") as f:
    for i in range(true_k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write (' %s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")

# CountVectorizer

In [8]:
# Importo las librerias necesarias.
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

'''
Creo el vectorizador - Función: countVectorizer.
Entreno el vectorizador con títulos candidatos a nivel de caracteres.
Creo las matrices de títulos candidatos y títulos encontrados.
'''
coun_vect = CountVectorizer(max_df=0.95,min_df=100,lowercase=True, analyzer="char", ngram_range=(1,3),max_features = 1000)
coun_vect.fit(titEncontrado)
count_candidatos = coun_vect.transform(titCandidato)
count_encontrados = coun_vect.transform(titEncontrado)
count_array = count_candidatos.toarray()
dfCountVec = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
dfCountVec

# Dimensiones: 
# encontrados 5286 rows × 300 columns
# candidatos 10105 rows × 300 columns

,,a,ac,au,c,cl,co,d,f,i,...,us,usi,ut,uto,v,va,vi,x,xa,xam
0,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15693,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Distancia cosenoidal

In [9]:
# Calculo de la similitud de vectores - Función: similitud coseno
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(count_candidatos, count_encontrados)

In [10]:
# Filas - Vectores - titulos candidatos
dist[:,0].shape

(15695,)

In [11]:
# Filas: titulos candidatos
# Columnas: titulos encontrados
dist.shape

(15695, 2898)

# BUSQUEDA DE TÍTULOS CANDIDATOS
## DISTANCIA COSENOIDAL

In [12]:
# Calculo la distancia entre títulos candidatos y títulos encontrados. Solo se guarda el valor mayor de similitud.
import numpy as np

candidatoList=[]
encotradoList=[]
distancia=[]
for i_cand in range(dist.shape[0]):
    i_enc = np.argmax(dist[i_cand])
    candidatoList.append(titCandidato[i_cand])
    encotradoList.append(titEncontrado[i_enc])
    distancia.append(dist[i_cand][i_enc])

# Creo un dataframe de Nuevos títulos encontrados y sus distancias cosenoidales.
dfNuevosCandidatosDistancia = pd.DataFrame({'candidato':candidatoList, 'encontrado': encotradoList, 'distancia': distancia})

In [13]:
dfNuevosCandidatosDistancia.to_csv('df_nuevosCandidatosDistancia.csv',index=False)
dfNuevosCandidatosDistancia.to_excel('df_nuevosCandidatosDistancia.xlsx',index=False)

# BUSQUEDA DE TÍTULOS CANDIDATOS
## Comparación de palabras

In [14]:
# Funcion para buscar titulos candidatos = titulos encontrados
def buscartitulo(tituloE, tituloC):
    titEncontrados = []
    for titulo in tituloE:
        for candidato in tituloC:
            if titulo == candidato:
                titEncontrados.append(candidato)
    return titEncontrados

    

In [15]:
# Creo los diccionarios de titulos candidatos y titulos encontrados.
encontradosTokens = encontrado.value_counts().head(50)
candidatoToken = candidato.value_counts().head(200)

In [16]:
# Convierte a lista los titulos
def listartitulos(tokens):
    listTit=[]
    for i,j in tokens.items():
        i = remove_stops(str(i),stops)
        listTit.append(i)
    return listTit

In [17]:
# Convierto titulos a listas de titulos.
candidatosList = listartitulos(candidatoToken)
encontradosList = listartitulos(encontradosTokens)

In [18]:
# Creo una variable que contiene los titulos candidatos = titulos encontrados
# Diccionario de titulos candidatos para la busqueda.
candidatosEncontrados = buscartitulo(encontradosList, candidatosList)

# Búsqueda de títulos en los títulos candidatos

In [19]:
# Buscar_candidatos: compara títulos candidatos con el diccionario te títulos candidatos=encontrados
def buscar_candidatos(encontrado, candidato, ide, inicio, fin):
    indiceEncontrados=[]
    for i in range(len(candidato)):
        for e in encontrado:
            if candidato[i] == e:
                indiceEncontrados.append((ide[i],candidato[i], inicio[i], fin[i]))
    return indiceEncontrados

In [20]:
# Lista con los títulos candidatos que son títulos de sección, su índice y ubicación
indicesCandidatos = buscar_candidatos(candidatosEncontrados ,dfTitCandidato.titulo, dfTitCandidato.ide, dfTitCandidato.inicio, dfTitCandidato.fin)

In [21]:
# Creo un dataframe de nuevos titulos encontrados y su ubicacion.
dfUbicacionNuevosEncontrados = pd.DataFrame(indicesCandidatos, columns=['ide', 'titulo', 'inicio', 'fin'])

In [22]:
dfUbicacionNuevosEncontrados.to_csv('df_UbicacionNuevosEncontrados.csv',index=False)
dfUbicacionNuevosEncontrados.to_excel('df_UbicacionNuevosEncontrados.xlsx',index=False)

# Busqueda de expedientes sin titulos localizados

In [23]:
# Guardo en dfExpNuevosTitulos los expedientes y su indice dentro del dataframe limpio (con todos los expedientes)

dfExpSinTitulosEncontrados = dfExpTitulosCandidatos
for id_e in dfUbicacionNuevosEncontrados.ide:
    dfExpSinTitulosEncontrados = dfExpSinTitulosEncontrados.drop(dfExpSinTitulosEncontrados[dfExpSinTitulosEncontrados['ide'] ==id_e].index)
    dfExpTitulosCandidatos.app
# dfExpNuevosTitulosEncontrados = pd.DataFrame(expConTitulosMayusculas, columns=['ide','expediente'])
# dfSinTitulosEncontrados = pd.DataFrame(expSinTitulosEncontrados, columns=['ide','expediente'])

In [24]:
pprint(dfExpSinTitulosEncontrados.expediente[1])

('JUZGADO NACIONAL DE TRABAJO 59 LAVALLE 1268 2ºP PERITO MEDICO DENUNCIA '
 'DOMICILIO ELECTRONICO - PRESENTA PERICIA - AUTORIZA EXPEDIENTE '
 '41300-201Señor Juez S/D. La Dra. Mabel Hilda Caputto Medica Legista MN '
 '61291, Medica Trabajo Mat. 3739, nombrada oficio Perito Medico autos '
 'caratulados " BARRERA ALEJANDRO ADRIAN c/MAILING GROUP SRL Y OTRO '
 's/ACCIDENTE – LEY ESPECIAL”. ratificando domicilio constituido San Jose '
 'Calazans 111 Planta Baja CABA, IVA Responsable INSCRIPTAcon CUIT numero '
 '27-10548071-2, Domicilio Electronico, VS respetuosamente digo: 27105480712, '
 '15-4422-183TELEFONO 1) Que vengo presente denunciar domicilio electronico, '
 'indicado encabezado, solicitando ser incluida nuevo sistema informatico '
 'efectos. 2) Que vengo informar examinado Sr. BARRERA ALEJANDRO ADRIAN '
 'procede elevar siguiente informe pericial. 1* Antecedentes autos Interes '
 'Medico Legal: Del INICIA DEMANDA apartado HECHOS: Manifiesta Sr. Actor '
 'BARRERA Alejandro Adrian,

# PRUEBAS

In [25]:
import seaborn as sns
import numpy as np

#sns.heatmap(dist)

In [26]:
import numpy
numpy.argmax(dist[0])



437

In [27]:
dist[0][864]

0.2752409412815902

In [28]:
titCandidato[0]
titEncontrado[864]

'rvan'

In [29]:
pruebaCandidatosList=['expedido','suscribe','acuerdo']
pruebaEncontradoList=['documental medico legal agregada expediente','observan pericia siguientes','podria hacerlo juez']
count_pruebaCandidato = coun_vect.transform(pruebaCandidatosList)
count_peurebaEncontrado = coun_vect.transform(pruebaEncontradoList)

In [30]:
distPrueba = cosine_similarity(count_pruebaCandidato, count_peurebaEncontrado)

In [31]:
# Calculo la distancia entre títulos candidatos y títulos encontrados. Solo se guarda el valor mayor de similitud.
import numpy as np

candidatoPruebaList=[]
encotradoPruebaList=[]
distanciaPrueba=[]
for i_cand in range(distPrueba.shape[0]):
    i_enc = np.argmax(distPrueba[i_cand])
    candidatoPruebaList.append(pruebaCandidatosList[i_cand])
    encotradoPruebaList.append(pruebaEncontradoList[i_enc])
    distanciaPrueba.append(distPrueba[i_cand][i_enc])

# Creo un dataframe de Nuevos títulos encontrados y sus distancias cosenoidales.
pd.DataFrame({'candidato':candidatoPruebaList, 'encontrado': encotradoPruebaList, 'distancia': distanciaPrueba})

,candidato,encontrado,distancia
0,expedido,documental medico legal agregada expediente,0.564114
1,suscribe,observan pericia siguientes,0.589256
2,acuerdo,podria hacerlo juez,0.596559


In [32]:
coun_vect = CountVectorizer(max_df=0.99,min_df=1,lowercase=True, analyzer="char", ngram_range=(1,3),max_features = 800)
coun_vect.fit(titEncontrado)
count_pruebaCandidato = coun_vect.transform(pruebaCandidatosList)
count_peurebaEncontrado = coun_vect.transform(pruebaEncontradoList)

In [33]:
distPrueba = cosine_similarity(count_pruebaCandidato, count_peurebaEncontrado)

In [34]:
# Calculo la distancia entre títulos candidatos y títulos encontrados. Solo se guarda el valor mayor de similitud.
import numpy as np

candidatoPruebaList=[]
encotradoPruebaList=[]
distanciaPrueba=[]
for i_cand in range(distPrueba.shape[0]):
    i_enc = np.argmax(distPrueba[i_cand])
    candidatoPruebaList.append(pruebaCandidatosList[i_cand])
    encotradoPruebaList.append(pruebaEncontradoList[i_enc])
    distanciaPrueba.append(distPrueba[i_cand][i_enc])

# Creo un dataframe de Nuevos títulos encontrados y sus distancias cosenoidales.
pd.DataFrame({'candidato':candidatoPruebaList, 'encontrado': encotradoPruebaList, 'distancia': distanciaPrueba})

,candidato,encontrado,distancia
0,expedido,documental medico legal agregada expediente,0.535706
1,suscribe,observan pericia siguientes,0.478913
2,acuerdo,podria hacerlo juez,0.485071
